In [1]:
import findspark
findspark.init('C:\\Spark\\spark-2.4.7-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("EDA") \
        .getOrCreate()

In [3]:
spark.sparkContext.setLogLevel("ERROR")

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType, IntegerType, DoubleType, DateType

In [5]:
df_review = spark.read.format('csv') \
.option("inferSchema", "True") \
.option("header", "True") \
.option("sep", ",") \
.option('escape','"')\
.option("escape","\"")\
.option("multiLine", "true")\
.option("encoding", "ISO-8859-1") \
.load('../data/airbnb_paris_data/reviews.csv')

In [6]:
df_review.show()

+----------+---------+-------------------+-----------+-------------+--------------------+
|listing_id|       id|               date|reviewer_id|reviewer_name|            comments|
+----------+---------+-------------------+-----------+-------------+--------------------+
|      2577|366217274|2019-01-02 00:00:00|   28047930|         Kate|Beautiful apartme...|
|      3109|123127969|2016-12-27 00:00:00|   12389804|       Sophie|The host canceled...|
|      3109|123274144|2016-12-28 00:00:00|   67553494|        Tom'S|The host canceled...|
|      3109|207127433|2017-10-28 00:00:00|   51636494|     Patricia|Tout s'est bien d...|
|      3109|208779822|2017-11-03 00:00:00|    4142888|     Patricia|Un petit nid foui...|
|      3109|234257457|2018-02-12 00:00:00|    3539452|    Dominique|The host canceled...|
|      3109|295840159|2018-07-24 00:00:00|    7415343|      Laurent|Appartement spaci...|
|      3109|297207211|2018-07-27 00:00:00|  202957965|     Eleonora|The host canceled...|
|      310

In [8]:
avg_length_of_stay_paris = 5.2
review_rate_modest = 0.5

In [10]:
df_review = df_review.groupBy("listing_id","date")\
                .count()\
                .withColumnRenamed("count", "review_day")\
                .withColumn('occupancy', F.round(avg_length_of_stay_paris *(F.col('review_day')/review_rate_modest),2))\
                .filter(F.col('date')>= "2016-01-01")

In [11]:
df_review.show()

+----------+-------------------+----------+---------+
|listing_id|               date|review_day|occupancy|
+----------+-------------------+----------+---------+
|      7397|2017-08-24 00:00:00|         1|     10.4|
|     27288|2016-02-24 00:00:00|         1|     10.4|
|     33993|2018-03-12 00:00:00|         1|     10.4|
|     38871|2017-12-06 00:00:00|         1|     10.4|
|     55348|2019-08-04 00:00:00|         1|     10.4|
|     59182|2017-02-28 00:00:00|         1|     10.4|
|     81106|2018-11-28 00:00:00|         1|     10.4|
|     86053|2018-01-05 00:00:00|         1|     10.4|
|    111270|2019-12-24 00:00:00|         1|     10.4|
|    120494|2016-01-24 00:00:00|         1|     10.4|
|    137112|2016-04-15 00:00:00|         1|     10.4|
|    166353|2018-07-04 00:00:00|         1|     10.4|
|    166370|2018-08-23 00:00:00|         1|     10.4|
|    180348|2017-01-10 00:00:00|         1|     10.4|
|    182744|2018-11-05 00:00:00|         1|     10.4|
|    215756|2018-10-25 00:00

In [12]:
df_listing = spark.read.format('csv') \
.option("inferSchema", "True") \
.option("header", "True") \
.option("sep", ",") \
.option('escape','"')\
.option("escape","\"")\
.option("multiLine", "true")\
.option("encoding", "ISO-8859-1") \
.load('../data/airbnb_paris_data/listings.csv')

In [13]:
host_table_temp = df_listing.select(['id','host_id','host_name','host_location','host_since',
                                'host_response_time','host_response_rate',
                                'host_is_superhost','host_acceptance_rate'])\
                                .dropDuplicates()

In [14]:
host_table = host_table_temp.drop('id')\
                                .dropDuplicates()

In [15]:
host_table.show()

+---------+--------------+--------------------+-------------------+------------------+------------------+-----------------+--------------------+
|  host_id|     host_name|       host_location|         host_since|host_response_time|host_response_rate|host_is_superhost|host_acceptance_rate|
+---------+--------------+--------------------+-------------------+------------------+------------------+-----------------+--------------------+
| 53469395| Miguel Ã�ngel|Majadahonda, Comu...|2016-01-05 00:00:00|               N/A|               N/A|                f|                 N/A|
| 31304304|        Badara|Miami Beach, Flor...|2015-04-15 00:00:00|               N/A|               N/A|                f|                 N/A|
| 35549967|      Penelope|Paris, ÃŽle-de-Fr...|2015-06-11 00:00:00|               N/A|               N/A|                f|                 N/A|
|300007169|          LÃ©a|Paris, ÃŽle-de-Fr...|2019-10-04 00:00:00|               N/A|               N/A|                f|       

In [16]:
listing_table = df_listing.select(['id','name', 'description',  'price', 'neighbourhood_group_cleansed', 
                          'latitude', 'longitude', 'property_type', 'accommodates', 
                          'bathrooms', 'bedrooms', 'beds'])\
                        .dropDuplicates()\
                        .withColumnRenamed("id", "id_listing")

In [17]:
listing_table.show()

+----------+--------------------+--------------------+-------+----------------------------+--------+---------+--------------------+------------+---------+--------+----+
|id_listing|                name|         description|  price|neighbourhood_group_cleansed|latitude|longitude|       property_type|accommodates|bathrooms|bedrooms|beds|
+----------+--------------------+--------------------+-------+----------------------------+--------+---------+--------------------+------------+---------+--------+----+
|    462721|COSY FAMILY APT M...|<b>The space</b><...|$180.00|                        null|48.84292|  2.29436|    Entire apartment|           4|     null|       3|   3|
|    675626|Nice 45m2 Paris -...|Appartement de 45...| $65.00|                        null|48.87902|  2.37342|    Entire apartment|           4|     null|       1|   2|
|    899349|Terrasse Facing E...|<b>The space</b><...|$150.00|                        null|48.85902|  2.30323|    Entire apartment|           2|     null| 

In [18]:
listing_temp= listing_table.select(['id_listing','price'])\
                            .withColumn("price", F.expr("substring(price,2,length(price)-1)"))\
                            .withColumn("price", F.col("price").cast(DoubleType()))

In [19]:
fact_revenue_table = df_review.join(listing_temp, (df_review.listing_id == listing_temp.id_listing))\
                            .join(host_table_temp, (df_review.listing_id == host_table_temp.id))\
                            .drop('id_listing')\
                            .drop('id')\
                            .withColumn('income_est', F.round(F.col('occupancy') *F.col('price'),2))\
                            .select(['date','listing_id', 'host_id','occupancy','income_est'])\
                            .withColumn('year', F.year('date')) \
                            .withColumn('month', F.month('date')) \
                            .withColumn('day', F.dayofmonth('date'))

In [20]:
fact_revenue_table.show()

+-------------------+----------+-------+---------+----------+----+-----+---+
|               date|listing_id|host_id|occupancy|income_est|year|month|day|
+-------------------+----------+-------+---------+----------+----+-----+---+
|2016-02-24 00:00:00|     27288|  77087|     10.4|    1300.0|2016|    2| 24|
|2017-02-28 00:00:00|     59182| 193906|     10.4|     592.8|2017|    2| 28|
|2018-08-14 00:00:00|    230112|1202318|     10.4|     728.0|2018|    8| 14|
|2017-12-31 00:00:00|    509357|2510200|     10.4|     728.0|2017|   12| 31|
|2016-01-03 00:00:00|    514402|2533682|     10.4|    1040.0|2016|    1|  3|
|2016-07-21 00:00:00|    552628|2715603|     10.4|     832.0|2016|    7| 21|
|2016-07-09 00:00:00|    605410|2998362|     10.4|    1040.0|2016|    7|  9|
|2017-07-26 00:00:00|    621314|2845167|     10.4|     416.0|2017|    7| 26|
|2019-06-05 00:00:00|    655276|3298295|     10.4|    1300.0|2019|    6|  5|
|2016-08-06 00:00:00|    686977|2798100|     10.4|    1664.0|2016|    8|  6|

In [21]:
time_table = fact_revenue_table.select(['date', 'year' , 'month', 'day']) \
                .withColumn('weekday', F.dayofweek('date'))\
                .dropDuplicates()

In [22]:
time_table.show()

+-------------------+----+-----+---+-------+
|               date|year|month|day|weekday|
+-------------------+----+-----+---+-------+
|2017-09-24 00:00:00|2017|    9| 24|      1|
|2019-06-24 00:00:00|2019|    6| 24|      2|
|2019-08-30 00:00:00|2019|    8| 30|      6|
|2018-08-10 00:00:00|2018|    8| 10|      6|
|2016-08-27 00:00:00|2016|    8| 27|      7|
|2019-02-16 00:00:00|2019|    2| 16|      7|
|2017-04-05 00:00:00|2017|    4|  5|      4|
|2016-05-20 00:00:00|2016|    5| 20|      6|
|2016-04-01 00:00:00|2016|    4|  1|      6|
|2017-08-22 00:00:00|2017|    8| 22|      3|
|2019-01-22 00:00:00|2019|    1| 22|      3|
|2019-08-19 00:00:00|2019|    8| 19|      2|
|2018-10-11 00:00:00|2018|   10| 11|      5|
|2017-02-13 00:00:00|2017|    2| 13|      2|
|2020-10-31 00:00:00|2020|   10| 31|      7|
|2017-10-09 00:00:00|2017|   10|  9|      2|
|2016-12-26 00:00:00|2016|   12| 26|      2|
|2019-07-11 00:00:00|2019|    7| 11|      5|
|2020-02-16 00:00:00|2020|    2| 16|      1|
|2016-10-3

In [26]:
time_table.write.parquet(os.path.join('../../output_tables', 'date_table'), 'overwrite')